# **VERSUSVIRUS - Team 306 - Real Time Bed Monitoring**

---



## **HOW LONG WILL A PATIENT STAY HOSPITALIZED?**

Regression on the duration of stay using an AutoML & OLS approach.

Data has been previously synthetized and designed to represent the distribution of Covid-19 patients admitted to Swiss hospitals and their medical conditions

In [0]:
# Load the Drive helper and mount
from google.colab import drive
import os
import pandas as pd
!pip install tpot
import tpot


drive.mount('/content/drive')
os.chdir("drive/My Drive/VersusVirus_Hackathon")

## Data Preparation

In [0]:
df = pd.read_excel('./TRAINING_DURATION_OF_STAY.xlsx')
df = pd.get_dummies(df, columns=['Admission Type', 'Gender'])

In [61]:
df.sample(5)

,Unnamed: 0,Age,Arrival Time,Arrival Date,Arrival hour range,Age bracket,Unique - Hospital,Spital,Latitude,Longitude,Station,PLZ,Ort,Full Address,zertifizierte Betten,Titel,Leitung Vorname,Leitung Name,Leitung Pflege,Anza hl,Status SGI Zertifizierung,Zertifikat von,Zertifikat bis,Gewicht,Hypertension,Cardiovascular Disease,Diabetes,Malignacy,Cerebrovascular disease,COPD,Chronoc kidney disease,Chronic liver disease,HIV infection,Fever,Fatigue,Dry cough,Anorexia,Myalgia,Dyspnea,Expectoration,Pharyngalgia,Diarrhea,Nausea,Dizziness,Headache,Vomiting,Abdominal Pain,Heart Rate (median),Respiratory Rate (median),Mean arterial pressure (median),# of pre_conditions,# of symptoms,Death,DUATION OF STAY,Discharge Time,Admission Type_ICU - Invasive,Admission Type_ICU - Non Invasive,Admission Type_Non-ICU,Gender_Female,Gender_Male
7577,7577,40,2020-03-12 14:02:49,2020-03-12,12-15,25-44,Kantonsspital Aarau (KSA)-Medizinische Intensi...,Kantonsspital Aarau (KSA),47.389074,8.060018,Medizinische Intensivstation,5001,Aarau,Kantonsspital Aarau (KSA) 5001 Aarau,8,Dr. med.,Marc,Michot,Gabriele Wessler,1.0,anerkannt,NaN,NaN,0.008256,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,86.17,22.00,91.18,0,2,0,5,2020-03-17 14:02:49,0,0,1,1,0
4304,4304,52,2020-03-12 02:53:23,2020-03-12,00-03,45-64,Ostschweiz. Kinderspital (OSKI)-Intensiv-Behan...,Ostschweiz. Kinderspital (OSKI),47.429958,9.394041,Intensiv-Behandlungsstation,9006,St. Gallen,Ostschweiz. Kinderspital (OSKI) 9006 St. Gallen,10,Dr. med.,Bjarte,Rogdo,Ruth Dutler,1.0,zertifiziert,27.03.18,26.03.23,0.010320,0,0,0,0,0,0,0,0,0,1,1,1,1,0,1,1,0,0,1,0,0,0,0,88.83,22.08,89.29,0,7,0,5,2020-03-17 02:53:23,0,0,1,0,1
9265,9265,29,2020-03-09 08:56:27,2020-03-09,04-07,25-44,Hôpital Fribourgeois - Site Fribourg (HFR)-Ser...,Hôpital Fribourgeois - Site Fribourg (HFR),46.694578,6.902950,Service des soins intensif et continus,1708,Fribourg,Hôpital Fribourgeois - Site Fribourg (HFR) 170...,19,NaN,Govind Oliver,Sridharan,Bernard Faglia,1.0,zertifiziert,04.03.19,03.03.24,0.019608,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,89.59,18.63,88.99,0,3,0,4,2020-03-13 08:56:27,0,0,1,0,1
1111,1111,28,2020-03-11 07:18:24,2020-03-11,04-07,25-44,Kantonsspital Bruderholz-Inst. für Anästhesie ...,Kantonsspital Bruderholz,47.527935,7.581290,Inst. für Anästhesie und Intensivmedizin,4101,Bruderholz,Kantonsspital Bruderholz 4101 Bruderholz,9,Dr. med.,Peter,Gerecke,Stephan Ness,1.0,anerkannt,NaN,NaN,0.009288,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,88.81,22.41,92.27,0,0,0,9,2020-03-20 07:18:24,0,1,0,0,1
6685,6685,75,2020-03-10 05:02:07,2020-03-10,04-07,65-84,UniversitätsSpital (USZ)-Institut für Intensiv...,UniversitätsSpital (USZ),47.376616,8.549177,Institut für Intensivmedizin,8091,Zürich,UniversitätsSpital (USZ) 8091 Zürich,64,Prof. Dr. med.,Reto,Schüpbach,Simone Stein,1.0,zertifiziert,31.08.18,30.08.23,0.066047,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,85.74,18.89,90.70,0,2,1,6,2020-03-16 05:02:07,1,0,0,0,1


## Define predictors and target variable

In [0]:
predictors = ['Age', 'Hypertension', 'Cardiovascular Disease', 'Diabetes',
       'Malignacy', 'Cerebrovascular disease', 'COPD',
       'Chronoc kidney disease', 'Chronic liver disease', 'HIV infection',
       'Fever', 'Fatigue', 'Dry cough ', 'Anorexia', 'Myalgia', 'Dyspnea',
       'Expectoration', 'Pharyngalgia', 'Diarrhea', 'Nausea', 'Dizziness',
       'Headache', 'Vomiting ', 'Abdominal Pain', 'Heart Rate (median)',
       'Respiratory Rate (median)', 'Mean arterial pressure  (median)',
       '# of pre_conditions', '# of symptoms', 'Admission Type_ICU - Invasive',
       'Admission Type_ICU - Non Invasive', 'Admission Type_Non-ICU',
       'Gender_Female', 'Gender_Male']

target = ['DUATION OF STAY']

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df[predictors], df[target].values[:, 0], test_size=0.25, random_state=42) #took 75% for training & 25% for testing

## Model Training

USE A TPOT PIPELINE TO DETERMINE BEST MODEL AMONGST MANY *(Running very long, adjust training parameters!)*

Evaluation Metric = RMSE. The error is calculated as the squareroot of the mean of the sum of all predicted duation of stay values - real duration of stay values for a given patient


$RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{d_i -f_i}{\sigma_i}\Big)^2}}$

In [0]:
# First import the relevant TPOT pipeline
from tpot import TPOTRegressor 
tpot = TPOTRegressor()
# Fits the training data on the 
tpot.fit(X_train, y_train)
# Exports best pipeline to a python file so it can be used
tpot.export('./my pipeline.py')

In [70]:
import numpy as np
print("RMSE after training and model selection for best regressor: ",np.sqrt(abs(tpot.score(X_test, y_test))))

MESE after training and model selection for best regressor:  2.1333814787472622



FIT A BENCHMARK OLS MODEL

In [68]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
ols1 = LinearRegression(normalize=True) #normalize inputs
ols1.fit(X_train, y_train)    # Fitting a linear model
y_pred = ols1.predict(X_test)
print("RMSE on test-set for OLS Benchmark Model",mean_squared_error(y_test, y_pred, squared=False))

RMSE on test-set for OLS Benchmark Model 2.1250818417065482


SCORE MODEL THAT CAME OUT OF THE TPOT PIPELINE

In [66]:
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.linear_model import LassoLarsCV, RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import Normalizer, RobustScaler
from sklearn.tree import DecisionTreeRegressor
from tpot.builtins import StackingEstimator

# Average CV score on the training set was: -4.811141924516791
exported_pipeline = make_pipeline(
    Normalizer(norm="l1"),
    SelectPercentile(score_func=f_regression, percentile=7),
    StackingEstimator(estimator=RidgeCV()),
    RobustScaler(),
    StackingEstimator(estimator=LassoLarsCV(normalize=False)),
    DecisionTreeRegressor(max_depth=3, min_samples_leaf=10, min_samples_split=7)
)

exported_pipeline.fit(X_train, y_train)
y_pred_automl = exported_pipeline.predict(X_test)
print("RMSE on test-set for AutoML best selected model: ",mean_squared_error(y_test, y_pred_automl, squared=False))

RMSE on test-set for AutoML best selected model:  2.1333814787472622


OLS MODEL SEEM TO BE ENOUGH

## OLS WITH VALIDATION SET

In [0]:
from sklearn.model_selection import train_test_split
df_sampled= df.sample(np.int(np.round(len(df)*0.95, 0))) # USE A RANDOM SUBSET AND LEAVE 5% AS VALIDATION SET 
df_val = df[~df.index.isin(df_sampled.index)]

X_train_, X_test_, y_train_, y_test_ = train_test_split(
    df_sampled[predictors], df_sampled[target].values[:, 0], test_size=0.25, random_state=42)

In [65]:
from sklearn.model_selection import cross_val_score
ols2 = LinearRegression(normalize=True)
ols2 = ols1.fit(X_train_, y_train_)
scores = cross_val_score(ols1, X_train_, y_train_, cv=5, scoring='neg_mean_squared_error')
print("Cross Validated - Test set validation score:" , np.sqrt(np.abs(scores.mean())))

Cross Validated - Test set validation score: 2.1724043554614143


RMSE on Validation Set

In [64]:
y_pred_val = ols2.predict(df_val[predictors])
print("RMSE on hold-out-set", mean_squared_error(df_val[target].values[:, 0], y_pred_val, squared=False)) #sqaured = False -> the error will not be squared to MSE hence, we ahave the RMSE

RMSE on hold-out-set 2.189016143081518


## Export Model for Deployment

In [0]:
import joblib 
# save the model to disk
filename = 'bed_duation_model_versus_virus_05APRIL2020.sav'
joblib.dump(ols2, filename)

In [0]:
#df_val[predictors].to_csv('./Deployment/INPUT_MODEL.csv')

**WE ARE ABLE TO PROVIDE A MODEL WITH A RMSE of ~2.2.**


